In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import keras
from sklearn import metrics as mt
from matplotlib import pyplot as plt

from keras.models import Sequential
from keras.layers import Input, Dense
from keras.layers import Reshape
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from keras.models import Model
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split
from sklearn import metrics as mt
from keras.layers.normalization import BatchNormalization

import os
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, StandardScaler

Using TensorFlow backend.


In [2]:
import librosa

In [3]:
X = []
y = []

In [7]:
genre_map = {}
df = pd.read_csv('data/fma_metadata/tracks.csv', encoding = 'latin1',low_memory=False)

In [8]:
for index, row in df.iterrows():
    if row.track_id is not np.nan and row.genre_top is not np.nan:
        try:
            genre_map[int(row.track_id)] = row.genre_top
        except:
            pass
        

In [9]:
not_found = 0
found = 0
for root, dirs, files in os.walk('data/'):
    for file in files:
        if '.mp3' in file: 
            try:
                if found<1000:
                    genre = genre_map[int(file[:6])]
                    x_raw, sr = librosa.load(root+'/'+'/'.join(dirs)+file, sr=22050)
                    mfcc = librosa.feature.mfcc(x_raw, sr=22050, n_mfcc=13, n_fft=512, hop_length=256)
                    X.append(mfcc)
                    y.append(genre)
                    found +=1 
            except:
                not_found += 1
        if found%25 is 0:
            print(found)
print(found)
print(not_found)

0
0
0
25
50
75
100
125
150
175
200
225
250
275
300
325
350
375
400
425
450
475
500
525
550
575
600
625
650
675
700
725
750
775
800
825
850
875
900
925
950
975
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1

In [14]:
X_test = [list(last) for last in [list(elem) for elem in [list(ind) for ind in X]]]
X_test = [list(elem) for elem in X_test]
print(type(X))
print(type(X[0]))
print(type(X[0][0]))
print(type(X[0][0][0]))

print(1)
print(type(X_test))
print(type(X_test[0]))
print(type(X_test[0][0]))
print(type(X_test[0][0][0]))

<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.float64'>
1
<type 'list'>
<type 'list'>
<type 'numpy.ndarray'>
<type 'numpy.float64'>


In [15]:
#2581 is the min length of the MFCCs that get read in
min_len = 10000000
for o_ind, o_elem in enumerate(X):
    for i_ind, i_elem in enumerate(o_elem):
        if(len(i_elem)<min_len):
            min_len = len(i_elem)
min_len

2581

In [43]:
X_copy = X.tolist()
for o_ind, o_elem in enumerate(X_copy):
    X_copy[o_ind] = list(X_copy[o_ind])
    for i_ind, i_elem in enumerate(o_elem):
        X_copy[o_ind][i_ind] = list(X_copy[o_ind][i_ind])
        X_copy[o_ind][i_ind] = X_copy[o_ind][i_ind][:500]
X = np.array(X_copy)

In [17]:
# import pickle
# output_X = open('X.pkl', 'wb')
# output_y = open('y.pkl', 'wb')
# pickle.dump(X, output_X)
# pickle.dump(y, output_y)
# output_X.close()
# output_y.close()

In [4]:
import pickle
input_X = open('X.pkl', 'rb')
input_y = open('y.pkl', 'rb')
X = pickle.load(input_X)
y = pickle.load(input_y)
input_X.close()
input_X.close()

In [5]:
np.unique(y)

array([u'Electronic', u'Experimental', u'Folk', u'Hip-Hop',
       u'Instrumental', u'International', u'Pop', u'Rock'],
      dtype='<U13')

In [53]:
#no runny
# print(X.shape)
# X_reshaped = np.reshape(X, (1000,13,500,1))
# X = X_reshaped

(1000, 13, 2581)


In [44]:
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_y)

In [45]:
print(X.shape)
X = np.reshape(X,(1000, 13, 1000,1))
print(X.shape)

(1000, 13, 1000)
(1000, 13, 1000, 1)


In [49]:
keras.backend.clear_session()

model = keras.models.Sequential()

model.add(Conv2D(3, (13, 10), input_shape=(13,1000, 1), strides=(1, 4)))
model.add(Activation("relu"))
print(model.output_shape)

model.add(Conv2D(15, (1, 10), strides=(1, 4)))
model.add(Activation("relu"))
print(model.output_shape)

model.add(Conv2D(65, (1, 10), strides=(1, 4)))
model.add(Activation("relu"))
print(model.output_shape)

model.add(Flatten())
print(model.output_shape)
model.add(Dense(dummy_y.shape[1]))
model.add(Activation("softmax"))
print(model.output_shape)

optimizer = keras.optimizers.SGD(1e-3)#lr=0.01, momentum=0.9, nesterov=True)
#optimizer = keras.optimizers.Adam()#lr=1e-5)#
model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X, dummy_y, epochs=100, verbose=1, validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])


(None, 1, 248, 3)
(None, 1, 60, 15)
(None, 1, 13, 65)
(None, 845)
(None, 8)
Train on 800 samples, validate on 200 samples
Epoch 1/100
800/800 [==============================] - 0s 475us/step - loss: 3.5347 - acc: 0.1937 - val_loss: 2.1209 - val_acc: 0.1400
Epoch 2/100
800/800 [==============================] - 0s 385us/step - loss: 2.0407 - acc: 0.1975 - val_loss: 2.0860 - val_acc: 0.1250
Epoch 3/100
800/800 [==============================] - 0s 360us/step - loss: 2.0263 - acc: 0.2038 - val_loss: 2.0873 - val_acc: 0.1250
Epoch 4/100
800/800 [==============================] - 0s 358us/step - loss: 2.0071 - acc: 0.2137 - val_loss: 2.1162 - val_acc: 0.1350
Epoch 5/100
800/800 [==============================] - 0s 390us/step - loss: 1.9972 - acc: 0.2200 - val_loss: 2.1014 - val_acc: 0.1600
Epoch 6/100
800/800 [==============================] - 0s 376us/step - loss: 1.9936 - acc: 0.2025 - val_loss: 2.1308 - val_acc: 0.1300
Epoch 7/100
800/800 [==============================] - 0s 371us/step

In [26]:
#oscars
keras.backend.clear_session()

l2_lambda = 0.0001

input_holder = Input(shape=(13,2581,1))

conv1 = Conv2D(32, 3, 3,
                input_shape=(13,2581,1),
                init='he_uniform',
                W_regularizer=l2(l2_lambda),
                activation='relu')(input_holder)

max1 = MaxPooling2D(pool_size=(2,2))(conv1)

conv2 = Conv2D(32, 3, 3,
                input_shape=(13,2581,1),
                init='he_uniform',
                W_regularizer=l2(l2_lambda),
                activation='relu')(max1)

max2 = MaxPooling2D(pool_size=(2,2))(conv2)

drop1 = Dropout(0.25)(max2)
flat1 = Flatten()(drop1)
dense1 = Dense(128,
              activation='relu',
              init='he_uniform',
              W_regularizer=l2(l2_lambda))(flat1)

drop2 = Dropout(0.5)(dense1)
dense2 = Dense(8,
              activation='sigmoid',
              init='glorot_uniform',
              W_regularizer=l2(l2_lambda))(drop2)

cnn = Model(input=input_holder,output=dense2)


#optimizer = keras.optimizers.SGD(1e-3)#lr=0.01, momentum=0.9, nesterov=True)
#optimizer = keras.optimizers.Adam()#lr=1e-5)#
cnn.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

cnn.fit(X, dummy_y, nb_epoch=5, verbose=1)

# loss = model.evaluate_generator(SampleLoader(val, batch_size=16), val.size, **params)
# loss = model.evaluate_generator(SampleLoader(test, batch_size=16), test.size, **params)
#Y = model.predict_generator(loader, test.size, pickle_safe=True, nb_worker=NB_WORKER, max_q_size=5)



/Users/luke/anaconda3/lib/python2.7/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), kernel_initializer="he_uniform", activation="relu", input_shape=(13, 2581,..., kernel_regularizer=<keras.reg...)`
  if sys.path[0] == '':
/Users/luke/anaconda3/lib/python2.7/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), kernel_initializer="he_uniform", activation="relu", input_shape=(13, 2581,..., kernel_regularizer=<keras.reg...)`
/Users/luke/anaconda3/lib/python2.7/site-packages/ipykernel_launcher.py:29: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_regularizer=<keras.reg..., kernel_initializer="he_uniform", activation="relu")`
/Users/luke/anaconda3/lib/python2.7/site-packages/ipykernel_launcher.py:35: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_regularizer=<keras.reg..., kernel_initializer="glorot_

ValueError: You are passing a target array of shape (1000, 1) while using as loss `categorical_crossentropy`. `categorical_crossentropy` expects targets to be binary matrices (1s and 0s) of shape (samples, classes). If your targets are integer classes, you can convert them to the expected format via:
```
from keras.utils import to_categorical
y_binary = to_categorical(y_int)
```

Alternatively, you can use the loss function `sparse_categorical_crossentropy` instead, which does expect integer targets.

['Electronic',
 'Experimental',
 'Folk',
 'Hip-Hop',
 'Instrumental',
 'International',
 'Pop',
 'Rock']

In [2]:
from keras.models import load_model

model = load_model('data/my_model.h5')

In [3]:
import coremltools
coreml_model = coremltools.converters.keras.convert(model)
coreml_model.author = 'ACLHOV'
coreml_model.license = 'SMU'
coreml_model.short_description = 'Slassifies song genre from extracted mfcc data from an audio track. Input: <13,TODO> ndarray of floats from an MFCC transform of a TODO second audio clip. Output: A one-hot Multiarray were the index with the biggest float value (0-1) is the recognized genre.'
coreml_model.save('CNNPredictGenre.mlmodel')

0 : conv2d_1_input, <keras.engine.topology.InputLayer object at 0x181eb1bb90>
1 : conv2d_1, <keras.layers.convolutional.Conv2D object at 0x181eb1bbd0>
2 : activation_1, <keras.layers.core.Activation object at 0x181eb92590>
3 : conv2d_2, <keras.layers.convolutional.Conv2D object at 0x181eb1be10>
4 : activation_2, <keras.layers.core.Activation object at 0x181eb1bdd0>
5 : conv2d_3, <keras.layers.convolutional.Conv2D object at 0x181eb33e10>
6 : activation_3, <keras.layers.core.Activation object at 0x181eb55a50>
7 : flatten_1, <keras.layers.core.Flatten object at 0x181eb333d0>
8 : dense_1, <keras.layers.core.Dense object at 0x181ebbe710>
9 : activation_4, <keras.layers.core.Activation object at 0x181ebd6710>
